# Multilevel Local Search for Traveling Salesman Problem with 20 cities

In [1]:
import numpy as np

In [2]:
np.random.seed(23)

# Generate 20 cities
city_list = np.array([i for i in range(1, 21)])
cities = city_list.copy()

# Shuffle cities
np.random.shuffle(cities)

print("Cities:")
print(city_list)

print("Cities in random order:")
print(cities)


# Level 1: Creating 10 clusters
level_1_clusters = [(cities[i], cities[i + 1]) for i in range(0, len(cities), 2)]

print("\nLevel 1 Clusters (10 clusters):")
print(level_1_clusters)

# Shuffle clusters
np.random.shuffle(level_1_clusters)

print("\nLevel 1 Clusters (shuffled):")
print(level_1_clusters)

# Level 2: Create 5 clusters out of the level 1 clusters
level_2_clusters = [(level_1_clusters[i], level_1_clusters[i + 1]) for i in range(0, len(level_1_clusters), 2)]

print("\nLevel 2 Clusters (5 clusters):")
print(level_2_clusters)

# Shuffle clusters
np.random.shuffle(level_2_clusters)

print("\nLevel 2 Clusters (shuffled):")
print(level_2_clusters)

Cities:
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
Cities in random order:
[ 1 11  3  2  5  6 15 16  4 17  8 12 13 14 18 19 10  9  7 20]

Level 1 Clusters (10 clusters):
[(1, 11), (3, 2), (5, 6), (15, 16), (4, 17), (8, 12), (13, 14), (18, 19), (10, 9), (7, 20)]

Level 1 Clusters (shuffled):
[(15, 16), (13, 14), (5, 6), (3, 2), (18, 19), (10, 9), (4, 17), (1, 11), (7, 20), (8, 12)]

Level 2 Clusters (5 clusters):
[((15, 16), (13, 14)), ((5, 6), (3, 2)), ((18, 19), (10, 9)), ((4, 17), (1, 11)), ((7, 20), (8, 12))]

Level 2 Clusters (shuffled):
[((15, 16), (13, 14)), ((18, 19), (10, 9)), ((7, 20), (8, 12)), ((4, 17), (1, 11)), ((5, 6), (3, 2))]


### Creating the clusters

The 20 cities will each be an element in a list. The cities are shuffled into a random order and pairs are created. These 10 pairs are shuffled and grouped into 5 clusters. The list is shuffled on final time.

In [3]:
num_cities = len(cities)

distance_matrix = np.random.randint(1, 5, size=(num_cities, num_cities))
distance_matrix = np.triu(distance_matrix, 1) + np.triu(distance_matrix, 1).T

np.fill_diagonal(distance_matrix, 0)

print("Distance matrix: ")
print(distance_matrix)

Distance matrix: 
[[0 4 1 2 2 1 4 4 3 4 4 1 3 3 2 3 3 1 2 4]
 [4 0 2 1 3 3 4 3 3 1 2 3 1 3 1 1 3 4 3 4]
 [1 2 0 3 2 4 4 3 2 1 4 3 4 4 3 4 2 2 3 1]
 [2 1 3 0 3 1 1 1 4 2 2 1 3 2 2 2 1 4 2 3]
 [2 3 2 3 0 2 3 3 1 2 1 3 2 4 2 1 4 4 3 1]
 [1 3 4 1 2 0 1 1 3 1 3 2 2 3 2 2 3 2 2 2]
 [4 4 4 1 3 1 0 4 4 3 2 3 4 3 1 4 1 1 4 2]
 [4 3 3 1 3 1 4 0 3 2 1 4 3 1 4 4 3 3 4 2]
 [3 3 2 4 1 3 4 3 0 1 3 3 4 2 3 4 3 2 2 2]
 [4 1 1 2 2 1 3 2 1 0 2 4 4 4 3 3 1 4 4 1]
 [4 2 4 2 1 3 2 1 3 2 0 2 3 3 2 3 1 1 2 3]
 [1 3 3 1 3 2 3 4 3 4 2 0 4 3 1 3 2 3 1 3]
 [3 1 4 3 2 2 4 3 4 4 3 4 0 1 1 1 1 2 1 1]
 [3 3 4 2 4 3 3 1 2 4 3 3 1 0 2 4 3 4 3 4]
 [2 1 3 2 2 2 1 4 3 3 2 1 1 2 0 1 2 2 2 1]
 [3 1 4 2 1 2 4 4 4 3 3 3 1 4 1 0 2 4 4 3]
 [3 3 2 1 4 3 1 3 3 1 1 2 1 3 2 2 0 1 3 4]
 [1 4 2 4 4 2 1 3 2 4 1 3 2 4 2 4 1 0 1 1]
 [2 3 3 2 3 2 4 4 2 4 2 1 1 3 2 4 3 1 0 3]
 [4 4 1 3 1 2 2 2 2 1 3 3 1 4 1 3 4 1 3 0]]


### Creating the distances between cities

This matrix defines the distance between the different cities. This will be used when calculating the fitness of a solution.

In [4]:
def calculate_cost(clusters, matrix, level):
    cities = []
    if level == 2:
        cities = flatten_from_level_2(clusters)
    elif level == 1:
        cities = combine_clusters(clusters)
    else:
        cities = clusters
    cost = 0
    for i in range(len(cities)):
        current_city = cities[i]
        if i == (len(cities)-1):
            next_city = cities[0]
        else:
            next_city = cities[i + 1]
        cost += matrix[current_city - 1, next_city - 1]
    return cost
    
def combine_clusters(clusters):
    return [city for pair in clusters for city in pair]

def flatten_from_level_2(clusters):
    level_1_flat = combine_clusters(clusters)
    return [city for pair in level_1_flat for city in pair]

def iterate(clusters, matrix, it, level):
    best_solution = clusters.copy()
    cc = calculate_cost(clusters, matrix, level)
    print(f'Solution : {clusters}, Cost : {cc}.')
    for i in range(it):
        random_index_1, random_index_2 = np.random.choice(len(clusters), 2, replace=False)
        print(f'The element with index {random_index_1} and the element with index {random_index_2} switch positions in the list (Using zero-based indexing)')
        clusters[random_index_1], clusters[random_index_2] = clusters[random_index_2], clusters[random_index_1]
        new_cost = calculate_cost(clusters, matrix, level)
        if new_cost <= cc:
            print(f'Solution : {clusters}, Cost : {new_cost}. The cost is equal to or better than the previous one. The new one will be considered the best solution')
            cc = new_cost
            best_solution = clusters.copy()
        else:
            print(f'Solution : {clusters}, Cost : {new_cost}. This solution is worse than the previous best solution. The previous one is still the best solution')
            clusters[random_index_1], clusters[random_index_2] = clusters[random_index_2], clusters[random_index_1]
    return best_solution, cc

### Functions

These functions can be used to move between levels, calculate the cost of solutions and perform the local serach.

### Finding a solution at level 2

In [5]:
level_2_iterations = 2
bestsol_level_2, cc_level_2 = iterate(level_2_clusters, distance_matrix, level_2_iterations, 2)
print(f'Best cost : {cc_level_2} \nBest solution : {bestsol_level_2}')

Solution : [((15, 16), (13, 14)), ((18, 19), (10, 9)), ((7, 20), (8, 12)), ((4, 17), (1, 11)), ((5, 6), (3, 2))], Cost : 44.
The element with index 4 and the element with index 0 switch positions in the list (Using zero-based indexing)
Solution : [((5, 6), (3, 2)), ((18, 19), (10, 9)), ((7, 20), (8, 12)), ((4, 17), (1, 11)), ((15, 16), (13, 14))], Cost : 48. This solution is worse than the previous best solution. The previous one is still the best solution
The element with index 3 and the element with index 1 switch positions in the list (Using zero-based indexing)
Solution : [((15, 16), (13, 14)), ((4, 17), (1, 11)), ((7, 20), (8, 12)), ((18, 19), (10, 9)), ((5, 6), (3, 2))], Cost : 42. The cost is equal to or better than the previous one. The new one will be considered the best solution
Best cost : 42 
Best solution : [((15, 16), (13, 14)), ((4, 17), (1, 11)), ((7, 20), (8, 12)), ((18, 19), (10, 9)), ((5, 6), (3, 2))]


### Going to level 1

In [6]:
level_1 = combine_clusters(bestsol_level_2)

In [7]:
level_1_iterations = 2
bestsol_level_1, cc_level_1 = iterate(level_1, distance_matrix, level_1_iterations, 1)
print(f'Best cost : {cc_level_1} \nBest solution : {bestsol_level_1}')

Solution : [(15, 16), (13, 14), (4, 17), (1, 11), (7, 20), (8, 12), (18, 19), (10, 9), (5, 6), (3, 2)], Cost : 42.
The element with index 7 and the element with index 8 switch positions in the list (Using zero-based indexing)
Solution : [(15, 16), (13, 14), (4, 17), (1, 11), (7, 20), (8, 12), (18, 19), (5, 6), (10, 9), (3, 2)], Cost : 39. The cost is equal to or better than the previous one. The new one will be considered the best solution
The element with index 8 and the element with index 2 switch positions in the list (Using zero-based indexing)
Solution : [(15, 16), (13, 14), (10, 9), (1, 11), (7, 20), (8, 12), (18, 19), (5, 6), (4, 17), (3, 2)], Cost : 41. This solution is worse than the previous best solution. The previous one is still the best solution
Best cost : 39 
Best solution : [(15, 16), (13, 14), (4, 17), (1, 11), (7, 20), (8, 12), (18, 19), (5, 6), (10, 9), (3, 2)]


### Going back to the original level

In [8]:
level_0 = combine_clusters(bestsol_level_1)

In [9]:
level_0_iterations = 2
bestsol_level_0, cc_level_0 = iterate(level_0, distance_matrix, level_0_iterations, 0)
print(f'Best cost : {cc_level_0} \nBest solution : {bestsol_level_0}')

Solution : [15, 16, 13, 14, 4, 17, 1, 11, 7, 20, 8, 12, 18, 19, 5, 6, 10, 9, 3, 2], Cost : 39.
The element with index 9 and the element with index 19 switch positions in the list (Using zero-based indexing)
Solution : [15, 16, 13, 14, 4, 17, 1, 11, 7, 2, 8, 12, 18, 19, 5, 6, 10, 9, 3, 20], Cost : 41. This solution is worse than the previous best solution. The previous one is still the best solution
The element with index 3 and the element with index 18 switch positions in the list (Using zero-based indexing)
Solution : [15, 16, 13, 3, 4, 17, 1, 11, 7, 20, 8, 12, 18, 19, 5, 6, 10, 9, 14, 2], Cost : 44. This solution is worse than the previous best solution. The previous one is still the best solution
Best cost : 39 
Best solution : [15, 16, 13, 14, 4, 17, 1, 11, 7, 20, 8, 12, 18, 19, 5, 6, 10, 9, 3, 2]


### Conclusion

Using two level with two iteration of local search at each level the best solution we get is (15, 16, 13, 14, 4, 17, 1, 11, 7, 20, 8, 12, 18, 19, 5, 6, 10, 9, 3, 2) with a total distance of 39 (including the distance back to the first city).